In [1]:
import sys
sys.path.append('..')


import numpy as np
from tqdm.notebook import tqdm
import h5py
from lib.Utilities import *
import os
from collections import Counter

In [2]:
# datafile = '/storage/ms5267@drexel.edu/precicecap_downloads/90_Patient_2023-03-21_12:19.h5'

annotation_group_names = []

# Directory containing the HDF5 files
directory_path = '/storage/ms5267@drexel.edu/precicecap_downloads/'

for filename in os.listdir(directory_path):
	if filename.endswith('.h5'):
		log_info(f"Processing {filename}")
		datafile = os.path.join(directory_path, filename)
		with h5py.File(datafile, 'r') as file:
			annotations_group = file['Annotations']
			for name, subgroup in annotations_group.items():
				annotation_data = file[f'Annotations/{name}'][:]
				if len(annotation_data)>0:
					annotation_group_names.extend(list(annotation_data[:, -2]))
				else:
					log_info("No annotation group")

print(Counter(annotation_group_names))

15:31:46 :	  Processing 85_Patient_2023-05-12_17:53.h5 

15:31:46 :	  Processing 59_Patient_2022-01-31_23:19.h5 

15:31:46 :	  Processing 74_Patient_2023-08-05_06:00.h5 

15:31:46 :	  Processing 110_Patient_2023_Sep_28__23_52_07_705708.h5 

15:31:46 :	  Processing 4_Patient_2022-02-05_08:59.h5 

15:31:46 :	  Processing 73_Patient_2017_Dec_18__11_19_55_297272.h5 

15:31:46 :	  Processing 34_Patient_2023-04-04_22:31.h5 

15:31:46 :	  Processing 53_Patient_2023-06-25_21:39.h5 

15:31:46 :	  Processing 101_Patient_2023_Nov_9__22_24_41_155873.h5 

15:31:46 :	  Processing 90_Patient_2023-03-21_12:19.h5 

15:31:46 :	  Processing 50_Patient_2023-06-12_21:10.h5 

15:31:46 :	  Processing 35_Patient_2023-04-03_19:51.h5 

15:31:46 :	  Processing 55_Patient_2023-06-13_00:47.h5 

15:31:46 :	  Processing 139_Patient_2024_Mar_4__7_32_51_662674.h5 

Counter({b'ECG': 5448, b'ABP': 3870, b'ART': 3644, b'ART2': 84, b'EEG': 66, b'Temp': 32, b'ART1': 26, b'na': 24, b'RR': 21, b'PLETH': 21, b'SpO2': 20, b'RE

In [23]:
# Run this to update the annotation file.
# Directory containing the HDF5 files
directory_path = '/storage/ms5267@drexel.edu/precicecap_downloads/'

keys_to_select = ['Waveforms/ART_na_Timestamps', 'Waveforms/ABP_na_Timestamps', 'Waveforms/ECG_II_Timestamps']
ann_keys = [b'ART', b'ABP', b'ECG']
ecg_abp_annotations = []
for filename in tqdm(os.listdir(directory_path)):
	if filename.endswith('.h5'):
		log_info(f"Processing {filename}")
		datafile = os.path.join(directory_path, filename)
		final_arr = np.array([['ID1', 'ID2', 'Session', 'Data_Type', 'Start_Time', 'End_Time', 'Signal_Type', 'Lead_Type']], dtype=str)
		with h5py.File(datafile, 'r') as file:
			# Process one data group at a time
			for ind, datagroup in enumerate(keys_to_select):
				# If the group does not exist in the file
				if datagroup not in file:
					continue
				
				timestamp = file[datagroup][:]

				annotations_group = file['Annotations']
			
				for name, subgroup in annotations_group.items():
					annotation_data = file[f'Annotations/{name}'][:]
					# If no annotation data need to print that there is no annotation for this file.
					if len(annotation_data)==0:
						log_info(f"No annotation group in {filename}")
						continue
					
					# print(ann_keys, annotation_data[:, -2])
					mask =  (annotation_data[:, -2] == ann_keys[ind])

					annotation_t = annotation_data[mask]

				# Create the annotation indices from the timestampe.
				idx_list=[]
				for ann in annotation_t:
					ann_start_ts = int(float(ann[2])*1e3)
					ann_end_ts = int(float(ann[3])*1e3)
					
					ann_start_idx, ann_end_idx = find_idx_from_ts(timestamp, ann_start_ts, ann_end_ts)
					
					idx_list.append([ann_start_idx, ann_end_idx])

				# print(np.array(idx_list).shape, np.array(annotation_t).shape)
				ann_array = np.hstack((np.array(idx_list), annotation_t)).astype(str)
				print(ann_array.shape, final_arr.shape)

				final_arr = np.vstack((final_arr, ann_array ))

			log_info(np.array(final_arr).shape)
			np.savetxt(f'../data/annotations/{filename}-annotations.csv', final_arr, delimiter=',', fmt='%s')

  0%|          | 0/17 [00:00<?, ?it/s]

16:08:18 :	  Processing 85_Patient_2023-05-12_17:53.h5 

(816, 8) (1, 8)
(186, 8) (817, 8)
16:08:23 :	  (1003, 8) 

16:08:23 :	  Processing 59_Patient_2022-01-31_23:19.h5 

(167, 8) (1, 8)
(167, 8) (168, 8)
(286, 8) (335, 8)
16:08:24 :	  (621, 8) 

16:08:24 :	  Processing 74_Patient_2023-08-05_06:00.h5 

(1071, 8) (1, 8)
(148, 8) (1072, 8)
16:08:30 :	  (1220, 8) 

16:08:30 :	  Processing 110_Patient_2023_Sep_28__23_52_07_705708.h5 

(904, 8) (1, 8)
(904, 8) (905, 8)
(1948, 8) (1809, 8)
16:08:36 :	  (3757, 8) 

16:08:36 :	  Processing 4_Patient_2022-02-05_08:59.h5 

(270, 8) (1, 8)
(203, 8) (271, 8)
16:08:36 :	  (474, 8) 

16:08:36 :	  Processing 73_Patient_2017_Dec_18__11_19_55_297272.h5 

(257, 8) (1, 8)
(117, 8) (258, 8)
16:08:36 :	  (375, 8) 

16:08:36 :	  Processing 34_Patient_2023-04-04_22:31.h5 

(302, 8) (1, 8)
(302, 8) (303, 8)
(837, 8) (605, 8)
16:08:37 :	  (1442, 8) 

16:08:37 :	  Processing 53_Patient_2023-06-25_21:39.h5 

(12, 8) (1, 8)
(8, 8) (13, 8)
16:08:37 :	  (21, 8) 


In [3]:
# # Run this to update the annotation file.
# # Directory containing the HDF5 files
# directory_path = '/storage/ms5267@drexel.edu/precicecap_downloads/'

# for filename in tqdm(os.listdir(directory_path)):
# 	if filename.endswith('.h5'):
# 		log_info(f"Processing {filename}")
# 		datafile = os.path.join(directory_path, filename)

# 		with h5py.File(datafile, 'r') as file:
# 			# First get the timestamp list for abp and ecg
# 			try:
# 				abp_timestamp = file['Waveforms/ART_na_Timestamps']
# 			except KeyError:
# 				abp_timestamp = file['Waveforms/ABP_na_Timestamps']	
			
# 			ecg_timestamp = file['Waveforms/ECG_II_Timestamps']
# 			abp_timestamp = abp_timestamp[:]
# 			ecg_timestamp = ecg_timestamp[:]

# 			annotations_group = file['Annotations']
			

# 			for name, subgroup in annotations_group.items():
# 				annotation_data = file[f'Annotations/{name}'][:]

# 				# If no annotation data need to print that there is no annotation for this file.
# 				if len(annotation_data)==0:
# 					log_info(f"No annotation group in {filename}")
# 					continue

# 				mask =  (annotation_data[:, -2] == b'ECG') | \
# 						(annotation_data[:, -2] == b'ABP') | \
# 						(annotation_data[:, -2] == b'ART') | \
# 						(annotation_data[:, -2] == b'ART1') | \
# 						(annotation_data[:, -2] == b'ART2')

# 				ecg_abp_annotations = annotation_data[mask]
			
# 			idx_list = []
# 			for ann in ecg_abp_annotations:
# 				ann_start_ts = int(float(ann[2])*1e3)
# 				ann_end_ts = int(float(ann[3])*1e3)
# 				# print(ann_start_ts, ann_end_ts, ann[4])
# 				if ann[4]==b'ECG':
# 					ann_start_idx, ann_end_idx = find_idx_from_ts(ecg_timestamp, ann_start_ts, ann_end_ts)
# 				else:
# 					ann_start_idx, ann_end_idx = find_idx_from_ts(abp_timestamp, ann_start_ts, ann_end_ts)

# 				idx_list.append([ann_start_idx, ann_end_idx])

# 			ann_array = np.hstack((np.array(idx_list), ecg_abp_annotations))
# 			log_info(ann_array.shape)
# 			np.savetxt(f'../data/annotations/{filename}-annotations.csv', ann_array, delimiter=',', fmt='%s')

19:36:15 :	  Processing 85_Patient_2023-05-12_17:53.h5 

19:36:20 :	  (1002, 8) 

19:36:20 :	  Processing 59_Patient_2022-01-31_23:19.h5 

19:36:21 :	  (620, 8) 

19:36:21 :	  Processing 74_Patient_2023-08-05_06:00.h5 

19:36:27 :	  (1219, 8) 

19:36:27 :	  Processing 110_Patient_2023_Sep_28__23_52_07_705708.h5 

19:36:33 :	  (3756, 8) 

19:36:33 :	  Processing 90_Patient_2023-03-21_19:57.h5 

19:36:36 :	  No annotation group in 90_Patient_2023-03-21_19:57.h5 

19:36:36 :	  (3756, 8) 

19:36:36 :	  Processing 4_Patient_2022-02-05_08:59.h5 

19:36:36 :	  (473, 8) 

19:36:36 :	  Processing 73_Patient_2017_Dec_18__11_19_55_297272.h5 

19:36:36 :	  (374, 8) 

19:36:36 :	  Processing 34_Patient_2023-04-04_22:31.h5 

19:36:37 :	  (1441, 8) 

19:36:37 :	  Processing 53_Patient_2023-06-25_21:39.h5 

19:36:37 :	  (20, 8) 

19:36:37 :	  Processing 101_Patient_2023_Nov_9__22_24_41_155873.h5 

19:36:41 :	  (606, 8) 

19:36:41 :	  Processing 90_Patient_2023-03-21_12:19.h5 

19:36:42 :	  (227, 8) 



In [86]:
abp_timestamp[4979380], abp_timestamp[4985252]

(1680689974005418, 1680690021002149)

In [56]:
abp_timestamp

array([1680650063562978, 1680650063570981, 1680650063578985, ...,
       1680697542722857, 1680697542730861, 1680697542738865])

In [4]:
# Directory containing the HDF5 files
directory_path = '/storage/ms5267@drexel.edu/precicecap_downloads/'

for filename in os.listdir(directory_path):
	if filename.endswith('.h5'):
		log_info(f"Processing {filename}")
		datafile = os.path.join(directory_path, filename)

		with h5py.File(datafile, 'r') as file:
			print(file['Waveforms'].keys())

17:51:20 :	  Processing 85_Patient_2023-05-12_17:53.h5 

<KeysViewHDF5 ['ABP_na', 'ABP_na_Timestamps', 'ECG_II', 'ECG_II_Timestamps', 'ICP_na', 'ICP_na_Timestamps', 'VideoIndex_na', 'VideoIndex_na_Timestamps']>
17:51:20 :	  Processing 59_Patient_2022-01-31_23:19.h5 

<KeysViewHDF5 ['ABP_na', 'ABP_na_Timestamps', 'ART_na', 'ART_na_Timestamps', 'CVP_na', 'CVP_na_Timestamps', 'ECG_II', 'ECG_II_Timestamps']>
17:51:20 :	  Processing 74_Patient_2023-08-05_06:00.h5 

<KeysViewHDF5 ['ABP_na', 'ABP_na_Timestamps', 'CO2_na', 'CO2_na_Timestamps', 'ECG_II', 'ECG_II_Timestamps', 'PLETH_na', 'PLETH_na_Timestamps', 'RESP_na', 'RESP_na_Timestamps']>
17:51:20 :	  Processing 110_Patient_2023_Sep_28__23_52_07_705708.h5 

<KeysViewHDF5 ['ABP_na', 'ABP_na_Timestamps', 'ART_na', 'ART_na_Timestamps', 'CVP_na', 'CVP_na_Timestamps', 'ECG_II', 'ECG_II_Timestamps', 'PLETH_na', 'PLETH_na_Timestamps', 'RESP_na', 'RESP_na_Timestamps']>
17:51:20 :	  Processing 90_Patient_2023-03-21_19:57.h5 

<KeysViewHDF5 ['ABP_na'

/storage/ms5267@drexel.edu/precicecap_downloads/
